In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import scipy
# import pandas
import torch
import utils
from utils import generate_state_space_Matern_23
from scipy import linalg
from utils import build_id_key_table
from model_Bayes_diffusion import Bayes_diffu_tensor
import tqdm

data_file = '../processed_data/beijing_15k.npy'
full_data = np.load(data_file, allow_pickle=True).item()

fold=0

# here should add one more data-loader class
data_dict = full_data['data'][fold]
data_dict['ndims'] = full_data['ndims']
data_dict['num_node'] = full_data['num_node']

data_dict['time_id_table'] = full_data['time_id_table']
data_dict['time_uni'] = full_data['time_uni']


hyper_dict={}

hyper_dict['epoch'] = 2
hyper_dict['ls'] = 0.1
hyper_dict['var'] = 0.1
hyper_dict['device'] = torch.device("cpu")
hyper_dict['R_U'] = 2 # dim of each node embedding
hyper_dict['c'] = 0.1 # diffusion rate
hyper_dict['a0']=1.0
hyper_dict['b0']=1.0


F,P_inf = utils.generate_state_space_Matern_23(data_dict,hyper_dict)

data_dict['F'] = F
data_dict['P_inf'] = P_inf

In [13]:
import numpy as np
import torch

a = [1,2,3]
b = [3,2,1]
ind_tr = data_dict["tr_ind"]
U= torch.rand(20,5,5,20)
U[a,:,:,b].shape

torch.Size([3, 5, 5])

In [15]:
torch.diag_embed(torch.rand(20,5),dim1=1).shape

torch.Size([20, 5, 5])

In [12]:
np.stack([a for i in range(5)],axis=1)

array([[1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3]])

In [2]:
torch.manual_seed(2)

N_T = len(data_dict['time_uni'])

model = Bayes_diffu_tensor(data_dict,hyper_dict)
test_rmse = model.model_test()
print('init state: test_rmse: %.4f '%(test_rmse))

EPOCH = 1
for epoch in tqdm.tqdm(range(EPOCH)):
    # forward
    for T in range(N_T):
        model.msg_update_U_llk_del(T)
        model.msg_update_U_llk(T)

        

        if T<N_T-1:
            model.msg_update_U_trans_del(T,mode='forward')
            # model.msg_update_U_trans_del(T,mode='backward')
            model.msg_update_U_trans_vec(T,mode='forward')
            # model.msg_update_U_trans(T,mode='forward')

        # assert utils.nan_check_1(model,T) is True
        # assert utils.neg_check_v(model,T) is True
    model.msg_update_U_trans_del(N_T-1,mode='backward')
    
    model.post_update_U()
    test_rmse = model.model_test()
    print('it: %d, test_rmse: %.4f '%(epoch,test_rmse))
    
    # backward 
    for T in reversed(range(N_T-1)):
        
        # model.msg_update_U_trans(T,mode='backward')
        model.msg_update_U_trans_vec(T,mode='backward')

        model.msg_update_U_llk_del(T)
        model.msg_update_U_llk(T)

        model.msg_update_U_trans_del(T,mode='backward')
        utils.nan_check_1(model,T)
        
    model.post_update_U()
    test_rmse = model.model_test()
    print('it: %d, test_rmse: %.4f '%(epoch,test_rmse))

KeyError: 'DAMPING'

In [3]:
T

50

In [4]:
time_gap = model.time_uni[T+1] - model.time_uni[T]
A_T_block = torch.block_diag(*([torch.matrix_exp(model.F * time_gap)]*model.R_U))
P_inf_block = torch.block_diag(*([model.P_inf]*model.R_U))
Q_T_block = P_inf_block - P_inf_block @ A_T_block @ P_inf_block.T

msg_m_l = model.msg_U_f_m_del[:,:,T].T.reshape(-1)
msg_v_l = model.msg_U_f_v_del[:,:,T].T.reshape(-1)
    
# msg from the right (from U_{T+1})
msg_m_r = model.msg_U_b_m_del[:,:,T+1].T.reshape(-1)
msg_v_r = model.msg_U_b_v_del[:,:,T+1].T.reshape(-1)

msg_m_r.requires_grad=True 
msg_v_r.requires_grad=True 
target_m = msg_m_r
target_v = msg_v_r

mu = (A_T_block @ msg_m_l.view(-1,1)).squeeze() # num_node * 1
sigma = torch.diag(msg_v_r) + Q_T_block + A_T_block @ torch.diag(msg_v_l) @ A_T_block.T
sample = msg_m_r

# print(sample)

# compute log-Z
dist = torch.distributions.multivariate_normal.MultivariateNormal(mu, sigma)
log_Z_trans = dist.log_prob(sample)

In [59]:
print(model.msg_U_f_m[:30,:,46].max())
print(model.msg_U_f_m[:30,:,47].max())
print(model.msg_U_llk_m_del[:,:,38].abs().max())

tensor(0.6097, dtype=torch.float64)
tensor(27.7729, dtype=torch.float64)
tensor(2.0244, dtype=torch.float64)


In [26]:
T = 46
eind_T = model.time_data_table_tr[T] # id of observed entries at this time-stamp
N_T = len(eind_T) 
ind_T = model.ind_tr[eind_T]
y_T = model.y_tr[eind_T].squeeze()

uid_table, _ = build_id_key_table( model.nmod,ind_T) # get the id of associated nodes
uid_list = []
for mode in range( model.nmod):
    uid_list_mode = [item + sum( model.ndims[:mode]) for item in uid_table[mode]]
    uid_list = uid_list + uid_list_mode
uid = np.array(uid_list).astype(np.int64)

# self.uid_table_T = uid_table
# self.ind_T = ind_T

U_llk_del_T = torch.cat([ model.msg_U_llk_m_del[:,:,T], model.msg_U_llk_v_del[:,:,T]]) # concat the m and v 
U_llk_del_T.requires_grad=True 

U_llk_del_T_m,U_llk_del_T_v =  model.arrange_U_llk(U_llk_del_T) # arrange U as mode-wise

E_z_del, E_z_2_del =  model.moment_product_U_del(ind_T,U_llk_del_T_m,U_llk_del_T_v) # first and second moment of CP-pred

model.msg_update_tau_del(T)
E_tau_del =  model.msg_tau_a_del_T[T]/ model.msg_tau_b_del_T[T]

log_Z = 0.5*N_T*torch.log(E_tau_del/(2*np.pi)) \
    -  0.5*E_tau_del* ( (y_T*y_T).sum() - 2* (y_T*E_z_del).sum() + E_z_2_del.sum())

In [33]:
U_llk_del_T_m

[tensor([[-2.0438, -1.8718],
         [ 0.3564,  0.5065],
         [-0.3009, -0.0341],
         [-3.6314, -1.0142],
         [-1.6843, -1.7258],
         [-2.4998, -2.8594],
         [-1.6391, -1.1571],
         [-2.1869, -1.7163],
         [-1.5967, -1.6505],
         [ 0.5131,  0.5052],
         [-1.1689, -1.2142],
         [-3.4670, -3.5964]], dtype=torch.float64, grad_fn=<SliceBackward>),
 tensor([[ 0.2203,  0.2832],
         [-2.0660, -2.6955],
         [-0.8595, -1.1216],
         [-0.9167, -1.0435],
         [ 0.3965,  0.3202],
         [-0.9552, -1.1956]], dtype=torch.float64, grad_fn=<SliceBackward>),
 tensor([[-2.0017, -1.9888],
         [-3.8004, -3.5413],
         [-3.0486, -2.9562],
         [-3.8260, -3.1128],
         [-6.6637, -6.4537],
         [-3.5730, -3.3011],
         [-6.0867, -2.8219],
         [-2.7000, -2.6690],
         [-5.3401, -5.0419],
         [-3.5458, -3.2852],
         [-4.4689, -4.2325],
         [-5.2201, -5.2612]], dtype=torch.float64, grad_fn=<Sli

In [25]:
y_T

tensor([-0.4085, -0.5081, -0.3995, -0.2412], dtype=torch.float64)

In [25]:
model.msg_U_f_m[:30,:,44].max()

tensor(-0.0139, dtype=torch.float64)

In [31]:
model.msg_U_b_m[:30,:,45].max()

tensor(0.9733, dtype=torch.float64)